# Ratings Prediction: Data Collection

### Objective:
To scrape the reviews of different laptops, Phones, Headphones, smart watches, Professional Cameras, Printers, monitors, Home theater, router from different e-commerce websites.


### Columns required:

* reviews of the product.

* rating of the product

In [ ]:
# Importing the neccessary libraries
import pandas as pd
import time
import requests
from  bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Scraping for ratings & reviews from Flipkart.com

In [ ]:
#path for chromedriver
driver = webdriver.Chrome('chromedriver.exe')
# to open the homepage of flipkart.com
url = "https://www.flipkart.com/"
driver.get(url)

time.sleep(3)

#clicking on cancel buttopn 
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()
search_items = ['Laptops','Monitors','Phones','Smart watches']

#empty lists
review_title = []
review_text = []
ratings = []

In [ ]:
#defining a function to scrape the data
def scrape():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            review_title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return
urls=[]
for i in search_items:
    # Find the search bar
    srcbar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srcbar.clear()
    srcbar.send_keys(i)
    # Clicking on the search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(2)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))
len(urls)    

In [ ]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
    # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrape()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrape()
    except: continue
len(ratings), len(review_text), len(review_title)     

In [ ]:
# Creating a dataframe
df = pd.DataFrame({})
df['Review_title'] = review_title
df['Review_text'] = review_text
df['Rating'] = ratings

In [ ]:
# Saving the data into csv file
df.to_csv("Flipkart_Reviews.csv")

# Scraping for ratings & reviews from Amazon.com

In [ ]:
# to open the homepage of amazon.com
url = "https://www.amazon.in/"
driver.get(url)

time.sleep(3)

search_items = ['Laptops','Phones','Headphones','Smart watches']
#empty lists
review_title = []
review_text = []
ratings = []

In [ ]:
for i in search_items:
    srcbar = driver.find_element_by_id("twotabsearchtextbox")
    srcbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  #clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
         try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            five_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(five_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        review_title.append(review.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 
len(ratings), len(review_title), len(review_text)

In [ ]:
# Creating a dataframe
df = pd.DataFrame({})
df['Review_title'] = review_title
df['Review_text'] = review_text
df['Rating'] = ratings

In [ ]:
# Saving the scraped data into csv file
df.to_csv("Amazon_Reviews.csv")

In [ ]:
# loading the flipkart and amazon reviews files to combine them together into a single csv file
Flipkart = pd.read_csv("Flipkart_Reviews.csv")
Amazon = pd.read_csv("Amazon_Reviews.csv")
Reviews = pd.concat([Amazon, Flipkart], ignore_index=True)
Reviews.drop(columns = 'Unnamed: 0', inplace=True)
Reviews

# Saving the final collected data to a csv file for further processing & building model

In [ ]:
Reviews.to_csv("Rating_Review_Data.csv")